In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
import optuna
import warnings

In [2]:
df_train = pd.read_csv('../input/tabularsep21-kfolddataset/train_10folds.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
submission_data = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

df_train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f112,f113,f114,f115,f116,f117,f118,claim,nan_count,kfold
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1,1,3
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0,0,8
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1,5,6
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1,2,6
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,10002.0,0.23049,1,8,9


In [3]:
features = [col for col in df_train.columns if col not in ('claim', 'kfold')]
features

['f1',
 'f2',
 'f3',
 'f4',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f27',
 'f28',
 'f29',
 'f30',
 'f31',
 'f32',
 'f33',
 'f34',
 'f35',
 'f36',
 'f37',
 'f38',
 'f39',
 'f40',
 'f41',
 'f42',
 'f43',
 'f44',
 'f45',
 'f46',
 'f47',
 'f48',
 'f49',
 'f50',
 'f51',
 'f52',
 'f53',
 'f54',
 'f55',
 'f56',
 'f57',
 'f58',
 'f59',
 'f60',
 'f61',
 'f62',
 'f63',
 'f64',
 'f65',
 'f66',
 'f67',
 'f68',
 'f69',
 'f70',
 'f71',
 'f72',
 'f73',
 'f74',
 'f75',
 'f76',
 'f77',
 'f78',
 'f79',
 'f80',
 'f81',
 'f82',
 'f83',
 'f84',
 'f85',
 'f86',
 'f87',
 'f88',
 'f89',
 'f90',
 'f91',
 'f92',
 'f93',
 'f94',
 'f95',
 'f96',
 'f97',
 'f98',
 'f99',
 'f100',
 'f101',
 'f102',
 'f103',
 'f104',
 'f105',
 'f106',
 'f107',
 'f108',
 'f109',
 'f110',
 'f111',
 'f112',
 'f113',
 'f114',
 'f115',
 'f116',
 'f117',
 'f118',
 'nan_count']

In [4]:
test['nan_count'] = test.isnull().sum(axis=1)
xtest = test[features]
mode = df_train[features].mode().iloc[0]
xtest = xtest.fillna(mode)

In [5]:
xtrain = df_train[features]
scaler = preprocessing.StandardScaler()

test[features] = scaler.fit_transform(xtest[features])
test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,nan_count
0,957919,1.747278,0.948442,-0.427147,-0.840645,0.095294,0.468778,-0.832226,-1.072016,-0.779770,...,-0.131863,-0.001763,-0.533905,-1.254592,0.194415,-0.124117,-0.629749,-0.634560,-0.250647,-0.442610
1,957920,0.909508,0.170491,-0.353351,2.503614,0.188642,-0.556304,-0.593130,0.528871,-0.524140,...,0.986689,-0.553983,-0.566362,0.986925,0.287187,-0.502827,0.022404,0.798596,-0.695993,-0.935935
2,957921,0.690577,0.661839,-0.515251,-0.731419,-0.063751,0.191682,0.818023,-0.160850,-0.609422,...,-0.708205,0.077831,-0.584945,-0.513748,-0.283915,-0.466968,-0.630140,0.552108,-1.045977,-0.442610
3,957922,-0.841062,0.324478,-0.474272,3.122008,0.426413,-0.039023,-0.979934,-0.136020,-0.795736,...,0.944134,-0.614268,-0.543467,-0.050570,-0.402660,-0.517696,2.560428,0.654220,-0.579111,-0.935935
4,957923,-0.240760,-2.435226,1.035997,0.701076,1.003687,-0.830404,1.108725,0.161900,-0.780536,...,0.061811,1.876614,-0.526856,-0.678597,-0.146556,0.355174,2.579090,-0.562310,-0.557469,-0.935935


In [6]:
df_train[features].head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,nan_count
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,5
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,2
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,10002.0,0.23049,8


In [13]:
xtrain.shape, df_train.claim.shape

((957919, 119), (957919,))

In [35]:
def run(trial):
    #optimize in one fold
    fold = 0
    xtrain = df_train[df_train.kfold != fold].reset_index(drop=True)
    xvalid = df_train[df_train.kfold == fold].reset_index(drop=True)
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = scaler.transform(xtrain[features])
    xvalid = scaler.transform(xvalid[features])
    
    learning_rate = trial.suggest_float('learning_rate', 1e-2, 0.8, log=True)  
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 0.9)
    max_depth = trial.suggest_int('max_depth', 1, 9)
    reg_lambda = trial.suggest_float('reg_lembda', 1e-5, 100.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 100.0)
    subsample = trial.suggest_float('sumsample', 0.1, 0.9)
    alpha = trial.suggest_int('alpha', 0, 100)
    #n_estimator = trial.suggest_int('n_estimator', 500, 4000)
    
    model = XGBClassifier(random_state = 0,
                          #n_estimator = n_estimator,
                          learning_rate = learning_rate,
                          colsample_bytree = colsample_bytree,
                          max_depth = max_depth,
                          reg_lambda = reg_lambda,
                          reg_alpha = reg_alpha,
                          subsample = subsample,
                          alpha = alpha, 
                          tree_method = 'gpu_hist',
                          gpu_id = 0,
                          predictor = 'gpu_predictor',
                          #boosting='gbdt',
                          #metric='multiclass',
                          eval_metric='mlogloss'
                         )
    model.fit(xtrain, ytrain)
    valid_preds = model.predict(xvalid)
    
    roc_auc = roc_auc_score(yvalid, valid_preds)
    return roc_auc

study = optuna.create_study(direction='maximize')   
study.optimize(run, n_trials=5)

[I 2021-09-14 18:34:21,340] A new study created in memory with name: no-name-51775d13-1a48-40fa-9dbd-41afc0622da2
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[I 2021-09-14 18:34:28,086] Trial 0 finished with value: 0.7730736313698521 and parameters: {'learning_rate': 0.08674869015674984, 'colsample_bytree': 0.894194483453287, 'max_depth': 7, 'reg_lembda': 96.69872679770177, 'reg_alpha': 89.60570782924681, 'sumsample': 0.2833840188066744, 'alpha': 35}. Best is trial 0 with value: 0.7730736313698521.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of la

In [36]:
best_params = study.best_params
best_params

{'learning_rate': 0.08674869015674984,
 'colsample_bytree': 0.894194483453287,
 'max_depth': 7,
 'reg_lembda': 96.69872679770177,
 'reg_alpha': 89.60570782924681,
 'sumsample': 0.2833840188066744,
 'alpha': 35}

In [37]:
final_preds = []
for fold in range(10):
    xtrain = df_train[df_train.kfold != fold].reset_index(drop=True)
    xvalid = df_train[df_train.kfold == fold].reset_index(drop=True)
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = scaler.transform(xtrain[features])
    xvalid = scaler.transform(xvalid[features])
    
    best_params = {'learning_rate': 0.08674869015674984,
                     'colsample_bytree': 0.894194483453287,
                     'max_depth': 7,
                     'reg_lembda': 96.69872679770177,
                     'reg_alpha': 89.60570782924681,
                     'sumsample': 0.2833840188066744,
                     'alpha': 35}
    
    model = XGBClassifier(random_state = 0,
                          #n_estimator = 500,
                          **best_params, 
                          tree_method = 'gpu_hist',
                          gpu_id = 0,
                          predictor = 'gpu_predictor',
                          eval_metric='mlogloss'
                         )
    model.fit(xtrain, ytrain)
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_preds.append(test_preds)
    print(fold, roc_auc_score(yvalid, valid_preds))

preds = np.mean(np.column_stack(final_preds), axis=1)
    

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:35:41] WARNING: ../src/learner.cc:573: 
Parameters: { "reg_lembda", "sumsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0 0.7729688808794686


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:35:50] WARNING: ../src/learner.cc:573: 
Parameters: { "reg_lembda", "sumsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


1 0.7721575348665891


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[18:36:41] WARNING: ../src/learner.cc:573: 
Parameters: { "reg_lembda", "sumsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


7 0.7728919237247462


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:36:50] WARNING: ../src/learner.cc:573: 
Parameters: { "reg_lembda", "sumsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


8 0.773525432758865


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:36:58] WARNING: ../src/learner.cc:573: 
Parameters: { "reg_lembda", "sumsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


9 0.7733823517762851


In [38]:
preds = np.mean(np.column_stack(final_preds), axis=1)
preds

array([1., 1., 1., ..., 1., 1., 1.])

In [39]:
submission_data.claim = preds
submission_data.to_csv('submission.csv', index=False)

In [40]:
submission_data.claim.value_counts().to_frame().T

,1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0
claim,491010,1380,453,257,139,93,58,36,29,13,6
